# Notebook Introduction

1. Data Preprocessing and Data Visualization
3. Build a Model
4. Model Evaluation

In [ ]:
pip install split-folders

In [ ]:
import os
import random
import splitfolders
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

import warnings
warnings.filterwarnings("ignore")

# Split Dataset

In [ ]:
splitfolders.ratio('../input/rice-image-dataset/Rice_Image_Dataset', output="data", seed=1337, ratio=(.8, 0.1,0.1))

In [ ]:
train_dir = './data/train'
val_dir = './data/val'
test_dir = './data/test'

# Data Preprocessing

In [ ]:

IMG_SIZE = (224,224)
train_data = image_dataset_from_directory(train_dir,
                                         label_mode = 'categorical',
                                         image_size = IMG_SIZE ,
                                         shuffle = True,
                                         batch_size = 64
                                         )
val_data = image_dataset_from_directory(val_dir,
                                         label_mode = 'categorical',
                                         image_size = IMG_SIZE ,
                                         shuffle = True,
                                         batch_size = 64
                                         )
test_data = image_dataset_from_directory(test_dir,
                                        label_mode = 'categorical',
                                        image_size = IMG_SIZE,
                                        shuffle = False,                                       
                                        batch_size = 64
                                        )

In [ ]:
labels = train_data.class_names
labels

# Data Visualization

In [ ]:

def plot_image(path , class_dir):
    target = path + class_dir
    random_path = random.choice(os.listdir(target))
    img = mpimg.imread(target + "/" + random_path)
    plt.imshow(img)
    plt.title(class_dir)
    plt.axis = False
#     print(f"Image_shape: {img.shape}")
    return img
    

In [ ]:
img = plot_image("./data/train/" , "Basmati")
img.shape

In [ ]:
for i in range(len(labels)):
    plt.subplot(2,4,i+1)
    img = plot_image("./data/train/",labels[i])

In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.layers.experimental import preprocessing

# data_augmentation = keras.Sequential([
#   preprocessing.RandomFlip('horizontal'),
#   preprocessing.RandomRotation(0.2),
#   preprocessing.RandomZoom(0.2),
#   preprocessing.RandomHeight(0.2),
#   preprocessing.RandomWidth(0.2) 
#   preprocessing.Rescale(1./255.)                                     
# ], name='data_augmentation')

# Model

In [ ]:
from tensorflow.keras import layers

INPUT_SHAPE = (224,224,3)

inputs = layers.Input(shape = INPUT_SHAPE)

x = layers.experimental.preprocessing.Rescaling(1/255) (inputs)
x = layers.Conv2D(32,3,padding = 'valid',activation='relu') (x)
x = layers.MaxPool2D(2) (x)

x = layers.Conv2D(64,3,padding = 'valid',activation = 'relu') (x)
x = layers.MaxPool2D(2) (x)

x = layers.Conv2D(128,3,padding = 'valid', activation = 'relu') (x)
x = layers.MaxPool2D(2) (x)

x = layers.Conv2D(128,3,padding = 'valid', activation = 'relu') (x)
x = layers.MaxPool2D(2) (x)

x = layers.Conv2D(256,3,padding = 'valid', activation = 'relu') (x)
x = layers.MaxPool2D(2) (x)

x = layers.GlobalAveragePooling2D() (x)

x = layers.Dense(256,activation = 'relu') (x)
x = layers.Dropout(0.2) (x)
x = layers.Dense(128,activation = 'relu') (x)
x = layers.Dropout(0.3) (x)
outputs = layers.Dense(5,activation = 'softmax') (x)

model_1 = tf.keras.models.Model(inputs,outputs)

model_1.summary()

In [ ]:
model_1.compile(loss = 'categorical_crossentropy',
               optimizer= tf.keras.optimizers.Adam(),
               metrics = ['accuracy'])


In [ ]:
history_1 = model_1.fit(train_data,
                       epochs = 5,
                       steps_per_epoch = len(train_data),
                       validation_data = val_data,
                       validation_steps = len(val_data)
                       )

In [ ]:
def plot_graphs(history):
    plt.figure(figsize = (8,8))
    epochs = range(1,len(history.history['loss'])+1)
    
    plt.subplot(2,1,1)
    plt.plot(epochs,history.history['accuracy'],'r',label = 'accuracy')
    plt.plot(epochs,history.history['val_accuracy'],'b',label = 'val_accuracy')
    plt.title("Training vs Validation accuracy")
    plt.legend()
    
    plt.subplot(2,1,2)
    plt.plot(epochs,history.history['loss'],'r',label = 'loss')
    plt.plot(epochs,history.history['val_loss'],'b',label = 'val_loss')
    plt.title("Training vs Validation loss")
    plt.legend()  

In [ ]:
plot_graphs(history_1)

In [ ]:
y_labels = []
imges = []
for images,labels in test_data.unbatch():
    y_labels.append(labels.numpy().argmax())

In [ ]:
model_1_pred_prob = model_1.predict(test_data)
model_1_pred = tf.argmax(model_1_pred_prob,axis = 1)

# Evaluation

In [ ]:
def evaluation_function(y_preds,y_true):
    accuracy = accuracy_score(y_true,y_preds)*100
    precision = precision_score(y_true,y_preds,average = 'weighted')*100
    recall = recall_score(y_true,y_preds,average = 'weighted')*100
    f1 = f1_score(y_true,y_preds,average = 'weighted')*100

    d={"Accuracy":accuracy,
     "Precision":precision,
     "Recall":recall,
     "F1-Score":f1}

    return d

In [ ]:
evaluation_function(model_1_pred,y_labels)

In [ ]:
print(classification_report(y_labels,model_1_pred))

In [ ]:
cm = confusion_matrix(y_labels,model_1_pred)
plt.figure(figsize = (10,7))
ax = sns.heatmap(cm,fmt='.2f', annot=True, cmap='Blues')

ax.set_title("Confusion_Metrics")
ax.set_xlabel("Predicted")
ax.set_ylabel("Actual")

ax.xaxis.set_ticklabels(test_data.class_names)
ax.yaxis.set_ticklabels(test_data.class_names)

plt.show()